In [1]:
import csv as csv
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
import random
import seaborn as sns
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, cross_validation

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.metrics import roc_curve, auc,confusion_matrix,classification_report,roc_auc_score
from pandas import DataFrame


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
officer = pd.read_csv("/Users/jeanroure/Documents/1- ESSEC&CENTRALE/14- Networks/Project/csv_panama_papers.2018-02-14/panama_papers.nodes.officer.csv", delimiter=",")
del officer['sourceID']
del officer ['countries']
del officer ['valid_until']


In [3]:
intermediaries = pd.read_csv("/Users/jeanroure/Documents/1- ESSEC&CENTRALE/14- Networks/Project/csv_panama_papers.2018-02-14/panama_papers.nodes.intermediary.csv", delimiter=",")
del intermediaries['sourceID']
del intermediaries ['countries']
del intermediaries ['valid_until']
             

In [4]:
entity = pd.read_csv("/Users/jeanroure/Documents/1- ESSEC&CENTRALE/14- Networks/Project/csv_panama_papers.2018-02-14/panama_papers.nodes.entity.csv", delimiter=",")
del entity['jurisdiction_description']
del entity['countries']
del entity['sourceID']
del entity['valid_until']


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
address = pd.read_csv("/Users/jeanroure/Documents/1- ESSEC&CENTRALE/14- Networks/Project/csv_panama_papers.2018-02-14/panama_papers.nodes.address.csv", delimiter=",")
del address['countries']
del address['sourceID']
del address['valid_until']

In [6]:
edges = pd.read_csv("/Users/jeanroure/Documents/1- ESSEC&CENTRALE/14- Networks/Project/csv_panama_papers.2018-02-14/panama_papers.edges.csv", delimiter=",")
del edges['sourceID']
del edges['valid_until']

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
officer.info(), intermediaries.info(), entity.info(), address.info(), edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238402 entries, 0 to 238401
Data columns (total 4 columns):
node_id          238402 non-null int64
name             238399 non-null object
country_codes    144789 non-null object
note             49 non-null object
dtypes: int64(1), object(3)
memory usage: 7.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14110 entries, 0 to 14109
Data columns (total 5 columns):
node_id          14110 non-null int64
name             14110 non-null object
country_codes    12598 non-null object
status           12621 non-null object
note             3 non-null object
dtypes: int64(1), object(4)
memory usage: 551.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213634 entries, 0 to 213633
Data columns (total 13 columns):
node_id               213634 non-null int64
name                  213630 non-null object
jurisdiction          213634 non-null object
country_codes         212844 non-null object
incorporation_date    213599 non-null object


(None, None, None, None, None)

In [42]:
officer.columns, intermediaries.columns, entity.columns, address.columns, edges.columns

(Index(['node_id', 'name', 'country_codes', 'note'], dtype='object'),
 Index(['node_id', 'name', 'country_codes', 'status', 'note'], dtype='object'),
 Index(['node_id', 'name', 'jurisdiction', 'country_codes',
        'incorporation_date', 'inactivation_date', 'struck_off_date',
        'closed_date', 'ibcRUC', 'status', 'company_type', 'service_provider',
        'note'],
       dtype='object'),
 Index(['node_id', 'name', 'address', 'country_codes', 'note'], dtype='object'),
 Index(['START_ID', 'TYPE', 'END_ID', 'link', 'start_date', 'end_date'], dtype='object'))

In [27]:
edges.TYPE.unique()

array(['registered_address', 'intermediary_of', 'officer_of'],
      dtype=object)

In [37]:
edges[edges['TYPE'] == 'officer_of']

,START_ID,TYPE,END_ID,link,start_date,end_date
218944,12000001,officer_of,10073324,shareholder of,19-NOV-1999,04-JUL-2000
218946,12000002,officer_of,10148386,shareholder of,30-MAR-2012,06-JUL-2012
218948,12000003,officer_of,10024966,shareholder of,14-JAN-2010,NaN
218950,12000004,officer_of,10004763,shareholder of,23-JUL-2012,NaN
218952,12000005,officer_of,10206741,shareholder of,13-SEP-2010,NaN
218954,12000006,officer_of,10100165,shareholder of,11-FEB-2002,NaN
218956,12000007,officer_of,10139840,shareholder of,22-APR-2010,NaN
218958,12000008,officer_of,10085853,shareholder of,03-JUN-2009,NaN
218960,12000009,officer_of,10085853,shareholder of,03-JUN-2009,NaN
218962,12000010,officer_of,10112597,shareholder of,01-JUL-2004,NaN


In [41]:
entity[entity['node_id']== 10142797] 

,node_id,name,jurisdiction,country_codes,incorporation_date,inactivation_date,struck_off_date,closed_date,ibcRUC,status,company_type,service_provider,note
143723,10142797,CULLODEN PROPERTIES LIMITED,BVI,GBR,14-JAN-2010,29-JUN-2015,31-OCT-2016,NaN,NaN,In transition,NaN,Mossack Fonseca,NaN


In [40]:
address[address['node_id']== 15006801] 

,node_id,name,address,country_codes,note
